In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 281.4/281.4 MB 4.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 22.8 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.3.2-py2.py3-none-any.whl size=281824028 sha256=6299394ba5918e7f71993621aca6ce5c6a95eaf4baa446f3820ffac3e707ab78
  Stored in directory: /root/.cache/pip/wheels/6c/e3/9b/0525ce8a69478916513509d43693511463c6468db0de237c86
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.8/310.8 MB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.4.0-py2.py3-none-any.whl size=311317130 sha256=ff52aba9a208ae6d96e13c9780ab9a3a0995ae6c87bfd7e18e751aae208babf8
  Stored in directory: /root/.cache/pip/wheels/7b/1b/4b/3363a1d04368e7ff0d408e57ff57966fcdf00583774e761327
Successfully built pyspark


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:

import findspark
findspark.init()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
from bs4 import BeautifulSoup
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.sql.functions import when
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from datetime import datetime
from pyspark.ml import Transformer
from pyspark.sql.types import FloatType
from pyspark.ml.param.shared import HasInputCol, HasOutputCol 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [ ]:
spark = SparkSession.builder.appName('Rating').getOrCreate()

### Load Data

In [ ]:
df1 = spark.read.csv("/content/drive/MyDrive/comments_final.csv",header=True
                      ,inferSchema=True)

In [ ]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- processed_text: string (nullable = true)
 |-- class: string (nullable = true)



In [ ]:
df1 = df1.withColumn('class',lit('fake'))

In [ ]:
df2 = df1.withColumn("class", when(df1['class'] == "negative","negative") \
      .when(df1['class'] == "neutral","negative") \
      .otherwise(df1['class']))
df2.show()

+---+--------------------+--------+
|_c0|      processed_text|   class|
+---+--------------------+--------+
|  0|    rất_hài sản_phẩm|positive|
|  1|sản_phẩm chất_lượ...|positive|
|  2|sản_phẩm chất_lượ...|positive|
|  3|cuộc_sống không_g...|positive|
|  4|viết nhận_xét quá...|negative|
|  5|hàng bị_lỗi màng ...|negative|
|  6|sản_phẩm rất_tốt ...|positive|
|  7|rất_hài sản_phẩm ...|positive|
|  8|băng xài ổn lực c...|positive|
|  9|sản_phẩm sản_phẩm...|negative|
| 10|hàng băng bị_cắt sài|negative|
| 11|mẫu_mã đẹp chất_l...|positive|
| 12|sản_phẩm không_đú...|negative|
| 13|hàng số_lượng đơn...|negative|
| 14|không_nói đừng sả...|negative|
| 15|dán vô rớt hoài v...|negative|
| 16|                băng|positive|
| 17|hàng đúng_hạn đón...|positive|
| 18|gọn bỏ_túi quảng_...|positive|
| 19|như_hình dùm chất...|positive|
+---+--------------------+--------+
only showing top 20 rows



In [ ]:
df2.count()

134231

In [ ]:
df2.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|103594|
|negative| 30637|
+--------+------+



### Precprocess

In [ ]:
df2 = df2.withColumn('length', length(df2['processed_text']))

In [ ]:
df2.show(5)

+---+--------------------+--------+------+
|_c0|      processed_text|   class|length|
+---+--------------------+--------+------+
|  0|    rất_hài sản_phẩm|positive|    16|
|  1|sản_phẩm chất_lượ...|positive|    85|
|  2|sản_phẩm chất_lượ...|positive|    85|
|  3|cuộc_sống không_g...|positive|    70|
|  4|viết nhận_xét quá...|negative|   543|
+---+--------------------+--------+------+
only showing top 5 rows



In [ ]:
df2.groupBy('class').mean().show()

+--------+-----------------+-----------------+
|   class|         avg(_c0)|      avg(length)|
+--------+-----------------+-----------------+
|positive| 65855.4977797942|46.68221132498021|
|negative|71373.80056794072|49.84929986617489|
+--------+-----------------+-----------------+



In [ ]:
data = df2.select('processed_text','class','length')

In [ ]:
data.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|    rất_hài sản_phẩm|positive|    16|
|sản_phẩm chất_lượ...|positive|    85|
|sản_phẩm chất_lượ...|positive|    85|
|cuộc_sống không_g...|positive|    70|
|viết nhận_xét quá...|negative|   543|
|hàng bị_lỗi màng ...|negative|    69|
|sản_phẩm rất_tốt ...|positive|    57|
|rất_hài sản_phẩm ...|positive|    23|
|băng xài ổn lực c...|positive|    32|
|sản_phẩm sản_phẩm...|negative|    90|
|hàng băng bị_cắt sài|negative|    20|
|mẫu_mã đẹp chất_l...|positive|    21|
|sản_phẩm không_đú...|negative|    30|
|hàng số_lượng đơn...|negative|    47|
|không_nói đừng sả...|negative|    23|
|dán vô rớt hoài v...|negative|    32|
|                băng|positive|     4|
|hàng đúng_hạn đón...|positive|    35|
|gọn bỏ_túi quảng_...|positive|    71|
|như_hình dùm chất...|positive|    26|
+--------------------+--------+------+
only showing top 20 rows



### Null

In [ ]:
null_data = data.filter(data['processed_text'].isNull())

In [ ]:
null_data.count()

33669

In [ ]:
data = data.filter(data['processed_text'].isNotNull())

### Duplicated

In [ ]:
data.dropDuplicates(['processed_text']).show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|aaaaa thích quá_t...|positive|    83|
|aaaaaaaaaaaaaaaaa...|positive|    41|
|          abhhgycycy|negative|    10|
|               acnes|positive|     5|
|              active|positive|     6|
|     active báo vọng|positive|    15|
|active việt nam a...|positive|    45|
|     active đóng_gói|positive|    15|
|      actived bị_cái|negative|    14|
|adaptor kích_thướ...|negative|    71|
|adaptor nóng thử ...|negative|    33|
|add quà tặng kèm ...|negative|    52|
|add vợ_chồng lên_...|positive|    32|
| addition tag nicely|positive|    19|
|ahas cũng_hơi ngạ...|positive|    67|
|       ahas việt nam|positive|    13|
|ahuhu lắp bị_đứt ...|positive|   189|
|       aiken rất_tốt|positive|    13|
|airpod bị_dính ẩn...|negative|    23|
|airpod máy case t...|positive|    31|
+--------------------+--------+------+
only showing top 20 rows



In [ ]:
data.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|    rất_hài sản_phẩm|positive|    16|
|sản_phẩm chất_lượ...|positive|    85|
|sản_phẩm chất_lượ...|positive|    85|
|cuộc_sống không_g...|positive|    70|
|viết nhận_xét quá...|negative|   543|
|hàng bị_lỗi màng ...|negative|    69|
|sản_phẩm rất_tốt ...|positive|    57|
|rất_hài sản_phẩm ...|positive|    23|
|băng xài ổn lực c...|positive|    32|
|sản_phẩm sản_phẩm...|negative|    90|
|hàng băng bị_cắt sài|negative|    20|
|mẫu_mã đẹp chất_l...|positive|    21|
|sản_phẩm không_đú...|negative|    30|
|hàng số_lượng đơn...|negative|    47|
|không_nói đừng sả...|negative|    23|
|dán vô rớt hoài v...|negative|    32|
|                băng|positive|     4|
|hàng đúng_hạn đón...|positive|    35|
|gọn bỏ_túi quảng_...|positive|    71|
|như_hình dùm chất...|positive|    26|
+--------------------+--------+------+
only showing top 20 rows



In [ ]:
data.count()

134231

In [ ]:
data.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|103594|
|negative| 30637|
+--------+------+



### Resampling

In [ ]:
from pyspark.sql.functions import col, explode, array, lit
major_df = data.filter(col("class") == 'positive')
minor_df = data.filter(col("class") == 'negative')
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))


ratio: 3


In [ ]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
combined_df_2.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|cuộc_sống không_g...|positive|    70|
|rất_hài sản_phẩm ...|positive|    23|
|                băng|positive|     4|
|cũng_rất dính tội...|positive|    34|
|băng_dính rất_tiệ...|positive|    32|
|         độ dính tốt|positive|    11|
|băng_dính dính tộ...|positive|    44|
|dính tốt bữa dán ...|positive|    39|
|                 bám|positive|     3|
| sản_phẩm dính kháng|positive|    19|
|chất_lượng tạm ổn...|positive|    30|
|đóng gói tốt hàng...|positive|    35|
| hình hiệu_quả không|positive|    19|
|dính thời rớt mặc...|positive|    34|
|đơn_giản độ bền s...|positive|    37|
|quá_chắc chắn tưở...|positive|    31|
|          mô_tả hàng|positive|    10|
|sản_phẩm đóng gói kỹ|positive|    20|
|dính dán kính cườ...|positive|    23|
|rất_tốt hạn_chế d...|positive|    25|
+--------------------+--------+------+
only showing top 20 rows



In [ ]:
combined_df_2.groupby('class').count().show()

+--------+-----+
|   class|count|
+--------+-----+
|positive|34449|
|negative|30637|
+--------+-----+



### Feature & Transform

In [ ]:
class BsTextExtractor(Transformer, HasInputCol, HasOutputCol):
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(BsTextExtractor, self).__init__() 
        kwargs = self._input_kwargs 
        self.setParams(**kwargs)
        
    @keyword_only
    def setParams(self, inputCol=None, outputCol=None): 
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    def _transform(self, dataset):
        def f(s):
            cleaned_text = BeautifulSoup(s).text 
            return cleaned_text
        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [ ]:
text_extractor = BsTextExtractor(inputCol="processed_text", outputCol="cleaned_text")
tokenizer = Tokenizer(inputCol='cleaned_text', outputCol='token_text')
stopremove= StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label', handleInvalid='keep')

In [ ]:
clean_up = VectorAssembler(inputCols =['tf_idf','length'],
                           outputCol='features')

In [ ]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,
                                  text_extractor,
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  clean_up])

In [ ]:
cleaner = data_prep_pipe.fit(combined_df_2)

In [ ]:
clean_data = cleaner.transform(combined_df_2)

In [ ]:
clean_data = clean_data.select('label','features')

In [ ]:
clean_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(20039,[277,533,9...|
|  0.0|(20039,[1,77,476,...|
|  0.0|(20039,[218,20038...|
|  0.0|(20039,[10,112,21...|
|  0.0|(20039,[50,112,12...|
|  0.0|(20039,[2,54,112,...|
|  0.0|(20039,[112,479,8...|
|  0.0|(20039,[2,50,65,7...|
|  0.0|(20039,[371,20038...|
|  0.0|(20039,[1,112,246...|
|  0.0|(20039,[4,14,58,2...|
|  0.0|(20039,[0,2,3,12,...|
|  0.0|(20039,[33,65,101...|
|  0.0|(20039,[26,50,112...|
|  0.0|(20039,[1,42,54,7...|
|  0.0|(20039,[47,100,10...|
|  0.0|(20039,[0,145,200...|
|  0.0|(20039,[1,12,20,8...|
|  0.0|(20039,[50,112,41...|
|  0.0|(20039,[50,66,521...|
+-----+--------------------+
only showing top 20 rows



In [ ]:
clean_data.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|  0.0|34449|
|  1.0|30637|
+-----+-----+



In [ ]:
(train, test) = clean_data.randomSplit([0.7,0.3])

## Build Model

### Naive Bayes

In [ ]:
nb = NaiveBayes()
prediction = nb.fit(train)
test_results = prediction.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time  
test_results.show()
print(train_time)

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(20039,[0,1,2,3,4...|[-1650.4987098192...|[1.0,2.2036748240...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-311.02182583160...|[1.0,2.4788237459...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-455.90784087959...|[0.99999984767774...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-233.68870554145...|[0.99999999999981...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-414.22459365381...|[0.99999784600106...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-133.66992069770...|[0.99999997283950...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-172.34210639873...|[0.99999999999993...|       0.0|
|  0.0|(20039,[0,1,2,3,4...|[-81.208736089573...|[0.99962966451059...|       0.0|
|  0.0|(20039,[0,1,2,3,5...|[-901.72554146786...|[1.0,7.5903264438...|       0.0|
|  0.0|(20039,[0

In [ ]:
test_results.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0| 1781|
|  0.0|       0.0| 8680|
|  1.0|       1.0| 7426|
|  1.0|       0.0| 1853|
+-----+----------+-----+



### Report

In [ ]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = test_results.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[8680. 1781.]
 [1853. 7426.]]


In [ ]:
acc_eva = MulticlassClassificationEvaluator()
acc = acc_eva.evaluate(test_results)
print('Accuracy of model: {}'.format(acc))

Accuracy of model: 0.8158639600626064


### Logistic Regression

In [ ]:
lg = LogisticRegression(maxIter=10, regParam=0.3)
pre_lg = lg.fit(train)
result_lg = pre_lg.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time 
print(train_time)

0:00:00.000082


In [ ]:
result_lg.groupBy('label', 'prediction').count().show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0| 1440|
|  0.0|       0.0| 9021|
|  1.0|       1.0| 7477|
|  1.0|       0.0| 1802|
+-----+----------+-----+



### Report

In [ ]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels_lg = result_lg.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels_lg = preds_and_labels_lg.select(['prediction','label'])

metrics_lg = MulticlassMetrics(preds_and_labels_lg.rdd.map(tuple))
print(metrics_lg.confusionMatrix().toArray())

/usr/local/lib/python3.9/dist-packages/pyspark/sql/context.py:157: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


[[9021. 1440.]
 [1802. 7477.]]


In [ ]:
acc_eva_lg = MulticlassClassificationEvaluator()
acc_lg = acc_eva_lg.evaluate(result_lg)
print('Accuracy of model: {}'.format(acc_lg))

Accuracy of model: 0.8355279201515133


#### Nhận xét:
- đối với việc sử dụng Pyspark, thời gian xử lý nhanh hơn, NB nhanh hơn về tốc độ xử lý so với logistic Regression
- Tuy nhiên kết quả thì logistic lại tốt hơn về độ chính xác

##### Như vậy Pyspark cho kết quả tốt hơn cả về thời gian xử lý lân độ chính xác